<a href="https://colab.research.google.com/github/MinamiNaoya/ExperimentTools/blob/main/md_simulate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDBファイルの取得

In [ ]:
!pip install biopython
import time
import urllib
from Bio.PDB import PDBList

pdb_ids = ['1ao6', '7jkz', '3i1y']

def download_file(url, dst_path):
    with urllib.request.urlopen(url) as web_file:
        with open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())

# AlphaFold
url = "https://alphafold.ebi.ac.uk/files/AF-P02768-F1-model_v4.pdb"

dst_path = "AF-P02768-F1-model_v4.pdb"
download_file(url, dst_path)

pdbl = PDBList()

for pdb_id in pdb_ids:
    pdbl.retrieve_pdb_file(pdb_id, pdir='pdb_files/')
    time.sleep(10)

# MD Simulation

In [ ]:
!conda install -c conda-forge openmm
!conda install -c conda-forge pdbfixer -y


### OpenMM setupはローカルでデプロイ

In [ ]:
import openmm as mm
import openmm.app as app
from openmm import unit
import pdbfixer

def preprocessing(pdb_file):
  # PDBFixerでPDBファイルを読み込む
  fixer = pdbfixer.PDBFixer(pdb_file)

  # 分子力場の設定
  forcefield = app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

  # 不要な構造の削除
  fixer.removeHeterogens()

  # 欠けている残基のチェック（欠損原子の確認のためにも必要）
  fixer.findMissingResidues()

  # タンパク質末端の欠けている残基を取り除く処理
  chains = list(fixer.topology.chains())
  keys = fixer.missingResidues.keys()
  for key in list(keys):
      chain = chains[key[0]]
      if key[1] == 0 or key[1] == len(list(chain.residues())):
          del fixer.missingResidues[key]

  # 非標準な残基が含まれているか確認、あれば標準的なものに置き換える
  fixer.findNonstandardResidues()
  fixer.replaceNonstandardResidues()

  # 欠けている原子の確認、あれば追加する
  fixer.findMissingAtoms()
  fixer.addMissingAtoms()

  # 水素原子の付与（pHを設定する）
  ph = 7.0
  fixer.addMissingHydrogens(ph)

  # 水ボックスの追加（力場、paddingの厚み、イオン濃度（デフォルトはNaCl））
  modeller = app.Modeller(fixer.topology, fixer.positions)
  modeller.addSolvent(forcefield, padding=1.0 * unit.nanometers, ionicStrength=0.15 * unit.molar)

  # 処理後の状態（トポロジー、原子の位置）をPDBファイルで出力
  top = modeller.getTopology()
  pos = modeller.getPositions()
  app.PDBFile.writeFile(top, pos, open(f'{pdb_file.replace(".pdb", "_preprocessed")}.pdb', 'w'))

## MD計算

In [ ]:
"""
以下はOpenMM setupで自動でスクリプトを作成できる。
"""
# This script was generated by OpenMM-Setup on 2024-06-08.

from openmm import *
from openmm.app import *
from openmm.unit import *

# Input Files

pdb = PDBFile('AF-P02768-F1-model_v4_preprocessed.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# System Configuration

nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 1.5*amu

# Integration Options

dt = 0.004*picoseconds
temperature = 300*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

# Simulation Options

steps = 1000000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('CUDA')
platformProperties = {'Precision': 'single'}
dcdReporter = DCDReporter('trajectory.dcd', 10000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 10000)

# Prepare the Simulation

print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform, platformProperties)
simulation.context.setPositions(positions)

# Minimize and Equilibrate

print('Performing energy minimization...')
simulation.minimizeEnergy()
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

# Simulate

print('Simulating...')
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.step(steps)